In [1]:
!pip install sqlalchemy

In [16]:
import sqlalchemy as db
db.__version__

'2.0.27'

In [31]:
engine = db.create_engine('sqlite:///product.db')

connection = engine.connect()

In [32]:
metadata = db.MetaData()

In [33]:
products = db.Table('products', metadata,
      db.Column('product_id', db.Integer, primary_key=True),
      db.Column('name', db.Text, ),
      db.Column('product_company', db.Text, ),
      db.Column('price', db.Integer, ),
)


In [34]:
client = db.Table(
    'client', metadata,
    db.Column('client_id', db.Integer, primary_key=True),
    db.Column('name', db.Text),
    db.Column('order_number', db.Integer),
    db.Column('order_product', db.Integer, db.ForeignKey('products.product_id'))
)


In [35]:
metadata.create_all(engine)

In [36]:
insertion = products.insert().values([
    {'name': 'bananas', 'product_company': 'united bananas', 'price': 8000},
    {'name': 'cucumbers', 'product_company': 'united cucumbers', 'price': 10000},
    {'name': 'tomatoes', 'product_company': 'united tomatoes', 'price': 11000},
    {'name': 'oranges', 'product_company': 'united otanges', 'price': 5000},
])

In [37]:
connection.execute(insertion)

In [38]:
insertion = client.insert().values([
    {'name': 'bananas_market', 'order_number': 1089, 'order_product': 1},
    {'name': 'oranges_market', 'order_number': 8000, 'order_product': 4},
    {'name': 'cucumbers_market', 'order_number': 152000, 'order_product': 2},
    {'name': 'tomatoes_market', 'order_number': 3100, 'order_product': 3},
])

In [39]:
connection.execute(insertion)

In [73]:
select_all = db.select(client)
print(select_all)
print('---------------------------------------')
result = connection.execute(select_all)
result.fetchall()

SELECT client.client_id, client.name, client.order_number, client.order_product 
FROM client
---------------------------------------


[(1, 'bananas_market', 1089, 1),
 (2, 'oranges_market', 8000, 4),
 (3, 'cucumbers_market', 152000, 2),
 (4, 'tomatoes_market', 3100, 3)]

In [72]:
#union
select_products = db.select(products)
select_client = db.select(client)
un = select_products.union(select_client)
print(un)
print('---------------------------------------')
un_result = connection.execute(un)
un_result.fetchall()

SELECT products.product_id, products.name, products.product_company, products.price 
FROM products UNION SELECT client.client_id, client.name, client.order_number, client.order_product 
FROM client
---------------------------------------


[(1, 'bananas', 'united bananas', 8000),
 (1, 'bananas_market', 1089, 1),
 (2, 'cucumbers', 'united cucumbers', 10000),
 (2, 'oranges_market', 8000, 4),
 (3, 'cucumbers_market', 152000, 2),
 (3, 'tomatoes', 'united tomatoes', 11000),
 (4, 'oranges', 'united otanges', 5000),
 (4, 'tomatoes_market', 3100, 3)]

In [71]:
# join
stmt = client.outerjoin(products, client.c.order_product == products.c.product_id)
stmt = stmt.select()
print(stmt)
print('---------------------------------------')
stmt_result = connection.execute(stmt)
stmt_result.fetchall()

SELECT client.client_id, client.name, client.order_number, client.order_product, products.product_id, products.name AS name_1, products.product_company, products.price 
FROM client LEFT OUTER JOIN products ON client.order_product = products.product_id
---------------------------------------


[(1, 'bananas_market', 1089, 1, 1, 'bananas', 'united bananas', 8000),
 (2, 'oranges_market', 8000, 4, 4, 'oranges', 'united otanges', 5000),
 (3, 'cucumbers_market', 152000, 2, 2, 'cucumbers', 'united cucumbers', 10000),
 (4, 'tomatoes_market', 3100, 3, 3, 'tomatoes', 'united tomatoes', 11000)]

In [70]:
# group by by / having
tmp_select = db.select(products, db.func.max(products.c.price).label("max")).group_by(products.c.price).having(db.func.max(products.c.price) > 10000)

print(tmp_select)
print('---------------------------------------')
tmp_select_result = connection.execute(tmp_select)
tmp_select_result.fetchall()

SELECT products.product_id, products.name, products.product_company, products.price, max(products.price) AS max 
FROM products GROUP BY products.price 
HAVING max(products.price) > :max_1
---------------------------------------


[(3, 'tomatoes', 'united tomatoes', 11000, 11000)]